In [30]:
!pip install pulp --quiet

In [31]:
# Step 1: Import necessary libraries and mount Google Drive
# ---------------------------------------------------------
import pandas as pd
import numpy as np
import pulp
from google.colab import drive
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os

warnings.filterwarnings("ignore", message="overflow encountered in exp")

print("--> Mounting Google Drive...")
try:
    drive.mount('/content/drive', force_remount=True)
    print("--> Drive mounted successfully!")
except Exception as e:
    print(f"ERROR: Could not mount drive. {e}")

--> Mounting Google Drive...
Mounted at /content/drive
--> Drive mounted successfully!


In [32]:
# Step 2: Define file paths and model parameters
# ---------------------------------------------------------
GDRIVE_BASE_PATH = '/content/drive/My Drive/Perceived Quality of Life Across Southeast Asian Cities/Datasets/'
COEF_FILEPATH = GDRIVE_BASE_PATH + 'full_bayesian_regression_results_with_marital_status.csv'
SURVEY_FILEPATH = GDRIVE_BASE_PATH + 'all_responses_coded.csv'
BASE_BUDGET = 20 # Base budget for the knapsack problem

# Base costs for each city
JAKARTA_COSTS = {
    'q3_1': 10, 'q3_2': 8, 'q1_2': 7, 'q4_7': 6, 'q1_4': 4,
    'q2_4': 5, 'q4_8': 3, 'q4_6': 2, 'q4_13': 2
}
PHNOM_PENH_COSTS = {
    'q3_1': 8, 'q3_2': 6, 'q1_2': 5, 'q4_7': 5, 'q1_4': 3,
    'q2_4': 5, 'q4_8': 3, 'q4_6': 2, 'q4_13': 2
}

# NEW GLOBAL DEFINITION: Policy definitions
POLICY_DEFINITIONS = { # Changed variable name to uppercase for global constant convention
    'q1_2': {'desc': 'I feel safe in this neighborhood', 'group': 'Space-Environment'},
    'q1_4': {'desc': 'This neighbourhood is a good place for children', 'group': 'Space-Environment'},
    'q3_1': {'desc': 'This neighbourhood has effective sanitary system', 'group': 'Space-Environment'},
    'q3_2': {'desc': 'This neighbourhood has an effective waste management system', 'group': 'Space-Environment'},
    'q4_7': {'desc': 'The healthcare facilities in my neighborhood are of good quality', 'group': 'Space-Environment'},
    'q2_4': {'desc': 'There are ample employment opportunities within accessible distance', 'group': 'Personal-Cohesion'},
    'q4_8': {'desc': 'I have saved money to invest for my extra income', 'group': 'Personal-Cohesion'},
    'q4_6': {'desc': 'In our neighbourhood neighbours look out for each other', 'group': 'Personal-Cohesion'},
    'q4_13': {'desc': 'I feel that I am not disadvantaged as compared to others', 'group': 'Personal-Cohesion'}
}

In [38]:
# Step 3: Define data loading and processing functions
# ---------------------------------------------------------------
# Ensure POLICY_DEFINITIONS is defined globally before this function.
# Example (put this near your JAKARTA_COSTS and PHNOM_PENH_COSTS):
# POLICY_DEFINITIONS = {
#    'q1_2': {'desc': 'I feel safe in this neighborhood', 'group': 'Space-Environment'},
#    # ... rest of your policy_definitions ...
# }

def load_and_process_coefficients(filepath: str, city_name: str) -> dict:
    try:
        df = pd.read_csv(filepath)
    except FileNotFoundError:
        print(f"ERROR: Coefficient file not found at {filepath}")
        return None

    # Use the GLOBAL POLICY_DEFINITIONS
    relevant_policy_terms_from_defs = list(POLICY_DEFINITIONS.keys())

    # Filter the DataFrame to only include relevant terms for the city or pooled, and the terms we need
    # This ensures we only process coefficients for the policies defined.

    # Do not set_index here for df_filtered, fetch rows by multiple criteria to avoid index issues
    city_df_filtered = df[
        (df['City'].isin([city_name, 'Pooled'])) &
        (df['term'].isin(relevant_policy_terms_from_defs))
    ]

    if city_df_filtered.empty:
        print(f"ERROR: No relevant coefficient data found for city {city_name} or Pooled for these policies.")
        return None

    coefs = {'policy_variables': {}}

    for term, details in POLICY_DEFINITIONS.items(): # Iterate through the GLOBAL POLICY_DEFINITIONS
        actual_term_row_data = None # Use a different variable name to avoid confusion

        # Prioritize city-specific coefficient over pooled
        city_specific_row_df = city_df_filtered[(city_df_filtered['City'] == city_name) & (city_df_filtered['term'] == term)]
        pooled_row_df = city_df_filtered[(city_df_filtered['City'] == 'Pooled') & (city_df_filtered['term'] == term)]

        if not city_specific_row_df.empty:
            actual_term_row_data = city_specific_row_df.iloc[0] # <-- FIX: Select the first row
        elif not pooled_row_df.empty:
            actual_term_row_data = pooled_row_df.iloc[0]       # <-- FIX: Select the first row

        if actual_term_row_data is not None:
            # Impact is max(0, estimate). This value is what's used in the objective function.
            impact_val = max(0, actual_term_row_data['estimate']) # <-- FIX: Use the selected row data
            # Uncertainty: Using std.error. Lower std.error is better, so we minimize it.
            uncertainty_val = actual_term_row_data['std.error'] # <-- FIX: Use the selected row data

            coefs['policy_variables'][term] = {
                'estimate': actual_term_row_data['estimate'], # Keep original estimate for reference
                'impact': impact_val,
                'uncertainty': uncertainty_val,
                'desc': details['desc'],
                'group': details['group']
            }
        else:
            print(f"WARNING: Policy term '{term}' not found specifically for {city_name} or in Pooled results. Skipping this policy for optimization.")
            continue
    return coefs

In [39]:
# Step 4: Define a function to find min/max values for epsilon constraints
# -----------------------------------------------------------------------
def get_objective_ranges(policy_data, budget):
    """
    Finds the approximate min/max values for each objective by solving
    single-objective knapsack problems. This helps set epsilon ranges.
    Returns: A dictionary with 'impact_range', 'need_range', 'uncertainty_range'
             each containing (min_val, max_val).
    """
    policy_terms = list(policy_data.keys())

    # Pre-calculate objective values for each policy
    policy_impacts = {p: policy_data[p]['impact'] for p in policy_terms}
    policy_needs = {p: policy_data[p]['need'] for p in policy_terms}
    # For uncertainty, we want to MINIMIZE, so we treat std_error as the value to be summed
    policy_uncertainties = {p: policy_data[p]['uncertainty'] for p in policy_terms}

    # --- Maximize Impact to find max_impact ---
    prob_max_impact = pulp.LpProblem("Max_Impact", pulp.LpMaximize)
    decision_vars = pulp.LpVariable.dicts("Policy", policy_terms, cat='Binary')
    prob_max_impact += pulp.lpSum([policy_impacts[p] * decision_vars[p] for p in policy_terms])
    prob_max_impact += pulp.lpSum([policy_data[p]['cost'] * decision_vars[p] for p in policy_terms]) <= budget
    prob_max_impact.solve(pulp.PULP_CBC_CMD(msg=0))
    max_impact = sum(policy_impacts[p] * decision_vars[p].varValue for p in policy_terms) if prob_max_impact.status == pulp.LpStatusOptimal else 0

    # --- Maximize Need to find max_need ---
    prob_max_need = pulp.LpProblem("Max_Need", pulp.LpMaximize)
    decision_vars = pulp.LpVariable.dicts("Policy", policy_terms, cat='Binary') # Re-declare decision_vars for clarity
    prob_max_need += pulp.lpSum([policy_needs[p] * decision_vars[p] for p in policy_terms])
    prob_max_need += pulp.lpSum([policy_data[p]['cost'] * decision_vars[p] for p in policy_terms]) <= budget
    prob_max_need.solve(pulp.PULP_CBC_CMD(msg=0))
    max_need = sum(policy_needs[p] * decision_vars[p].varValue for p in policy_terms) if prob_max_need.status == pulp.LpStatusOptimal else 0

    # --- Minimize Uncertainty to find min_uncertainty ---
    # Note: If no policies are selected, uncertainty is 0. So min_uncertainty will often be 0.
    prob_min_uncertainty = pulp.LpProblem("Min_Uncertainty", pulp.LpMinimize)
    decision_vars = pulp.LpVariable.dicts("Policy", policy_terms, cat='Binary') # Re-declare decision_vars
    prob_min_uncertainty += pulp.lpSum([policy_uncertainties[p] * decision_vars[p] for p in policy_terms])
    prob_min_uncertainty += pulp.lpSum([policy_data[p]['cost'] * decision_vars[p] for p in policy_terms]) <= budget
    prob_min_uncertainty.solve(pulp.PULP_CBC_CMD(msg=0))
    min_uncertainty = sum(policy_uncertainties[p] * decision_vars[p].varValue for p in policy_terms) if prob_min_uncertainty.status == pulp.LpStatusOptimal else 0 # If nothing chosen, uncertainty is 0.

    # --- To find min_impact and min_need ---
    # The minimum impact/need, when selecting policies, would generally be 0 if no policies are chosen,
    # or the smallest positive value if at least one policy must be chosen.
    # For a general knapsack, if we allow an empty set (total cost 0), then min_impact/need would be 0.
    min_impact = 0.0 # Assuming 0 if no policies selected
    min_need = 0.0   # Assuming 0 if no policies selected

    # To find max_uncertainty (worst case, i.e., sum of uncertainties of all chosen policies)
    # This happens when we select policies that sum up to the maximum uncertainty allowed by the budget.
    # This is equivalent to maximizing the uncertainty objective, similar to max_impact/max_need
    prob_max_uncertainty = pulp.LpProblem("Max_Uncertainty", pulp.LpMaximize)
    decision_vars = pulp.LpVariable.dicts("Policy", policy_terms, cat='Binary')
    prob_max_uncertainty += pulp.lpSum([policy_uncertainties[p] * decision_vars[p] for p in policy_terms])
    prob_max_uncertainty += pulp.lpSum([policy_data[p]['cost'] * decision_vars[p] for p in policy_terms]) <= budget
    prob_max_uncertainty.solve(pulp.PULP_CBC_CMD(msg=0))
    max_uncertainty_val = sum(policy_uncertainties[p] * decision_vars[p].varValue for p in policy_terms) if prob_max_uncertainty.status == pulp.LpStatusOptimal else 0.0

    return {
        'impact_range': (min_impact, max_impact),
        'need_range': (min_need, max_need),
        'uncertainty_range': (min_uncertainty, max_uncertainty_val)
    }

In [40]:
# Step 5: Define the multi-objective optimization function using Epsilon-Constraint
# ----------------------------------------------------------------------------------
def solve_multi_objective_city(city_name: str, policy_data: dict, budget: float,
                               epsilon_need_values: list, epsilon_uncertainty_values: list):
    """
    Solves the multi-objective knapsack problem for a given city using the epsilon-constraint method.
    It maximizes Impact, subject to budget and epsilon constraints on Need and Uncertainty.

    Args:
        city_name (str): Name of the city.
        policy_data (dict): Dictionary of policies with their costs, impacts, needs, and uncertainties.
        budget (float): Total budget for policies.
        epsilon_need_values (list): List of epsilon values to iterate for Need (as constraint).
        epsilon_uncertainty_values (list): List of epsilon values to iterate for Uncertainty (as constraint).

    Returns:
        pd.DataFrame: A DataFrame containing all Pareto optimal solutions found,
                      with chosen policies and objective values.
    """
    policy_terms = list(policy_data.keys())

    # Store unique Pareto solutions
    all_raw_solutions = []

    # Iterate through all combinations of epsilon constraints for Need and Uncertainty
    # The primary objective will be Impact (Maximized)

    # Need constraint (Z2 >= epsilon_need)
    for epsilon_need in epsilon_need_values:
        # Uncertainty constraint (Z3 <= epsilon_uncertainty)
        for epsilon_uncertainty in epsilon_uncertainty_values:
            prob = pulp.LpProblem(f"MOKP_{city_name}_EpsNeed{epsilon_need:.2f}_EpsUncert{epsilon_uncertainty:.2f}", pulp.LpMaximize)
            decision_vars = pulp.LpVariable.dicts("Policy", policy_terms, cat='Binary')

            # Objective: Maximize Total Aspiration Impact (Z1)
            prob += pulp.lpSum([policy_data[term]['impact'] * decision_vars[term] for term in policy_terms]), "Total_Aspiration_Impact"

            # Constraint 1: Budget Constraint
            prob += pulp.lpSum([policy_data[term]['cost'] * decision_vars[term] for term in policy_terms]) <= budget, "Budget_Constraint"

            # Constraint 2: Epsilon constraint for Need (Z2 >= epsilon_need)
            prob += pulp.lpSum([policy_data[term]['need'] * decision_vars[term] for term in policy_terms]) >= epsilon_need, f"Need_Constraint_GE_{epsilon_need:.2f}"

            # Constraint 3: Epsilon constraint for Uncertainty (Z3 <= epsilon_uncertainty)
            # Note: Uncertainty is minimized, so we constrain it to be *less than or equal to* epsilon_uncertainty
            prob += pulp.lpSum([policy_data[term]['uncertainty'] * decision_vars[term] for term in policy_terms]) <= epsilon_uncertainty, f"Uncertainty_Constraint_LE_{epsilon_uncertainty:.2f}"

            prob.solve(pulp.PULP_CBC_CMD(msg=0))

            if prob.status == pulp.LpStatusOptimal:
                # Calculate objective values for the current solution
                current_impact = sum(policy_data[term]['impact'] * decision_vars[term].varValue for term in policy_terms)
                current_need = sum(policy_data[term]['need'] * decision_vars[term].varValue for term in policy_terms)
                current_uncertainty = sum(policy_data[term]['uncertainty'] * decision_vars[term].varValue for term in policy_terms)
                current_cost = sum(policy_data[term]['cost'] * decision_vars[term].varValue for term in policy_terms)

                chosen_policies_detail = [
                    {'Variable': term,
                     'Policy': policy_data[term]['desc'],
                     'Group': policy_data[term]['group'],
                     'Cost': policy_data[term]['cost'],
                     'Impact': policy_data[term]['impact'],
                     'Need': policy_data[term]['need'],
                     'Uncertainty': policy_data[term]['uncertainty']}
                    for term in policy_terms if decision_vars[term].varValue == 1
                ]

                # To ensure uniqueness based on policy selection, not just objective values
                chosen_policy_variables_tuple = tuple(sorted([p['Variable'] for p in chosen_policies_detail]))

                # Add this solution to the list
                all_raw_solutions.append({
                    'City': city_name,
                    'Epsilon_Need_Applied': epsilon_need, # The epsilon value used for this run
                    'Epsilon_Uncertainty_Applied': epsilon_uncertainty, # The epsilon value used for this run
                    'Total_Impact': current_impact,
                    'Total_Need': current_need,
                    'Total_Uncertainty': current_uncertainty,
                    'Total_Cost': current_cost,
                    'Chosen_Policy_Variables_Tuple': chosen_policy_variables_tuple, # For uniqueness check
                    'Chosen_Policies_Detail': chosen_policies_detail # Detailed list for reporting
                })

    all_solutions_df = pd.DataFrame(all_raw_solutions)

    if all_solutions_df.empty:
        print(f"No feasible solutions found for {city_name} under the given epsilon ranges and budget.")
        return pd.DataFrame() # Return empty if no solutions found

    # --- Filter for Non-dominated (Pareto Optimal) Solutions ---
    # A solution is non-dominated if no other solution is better in ALL objectives
    # (Total_Impact and Total_Need are Maximized, Total_Uncertainty is Minimised)

    # Sort to make dominance checking more efficient (e.g., sort by max objectives descending, min objective ascending)
    all_solutions_df = all_solutions_df.sort_values(by=['Total_Impact', 'Total_Need', 'Total_Uncertainty'],
                                                    ascending=[False, False, True]).reset_index(drop=True)

    pareto_front_solutions = []

    for i in range(len(all_solutions_df)):
        is_dominated = False
        sol_i = all_solutions_df.iloc[i]

        for j in range(len(all_solutions_df)):
            sol_j = all_solutions_df.iloc[j]
            if i == j: # Don't compare a solution to itself
                continue

            # Check if sol_j dominates sol_i
            # sol_j dominates sol_i if:
            # (sol_j is >= sol_i in Impact AND sol_j is >= sol_i in Need AND sol_j is <= sol_i in Uncertainty)
            # AND (sol_j is STRICTLY BETTER in at least one of these: Impact OR Need OR Uncertainty)

            dom_impact = sol_j['Total_Impact'] >= sol_i['Total_Impact']
            dom_need = sol_j['Total_Need'] >= sol_i['Total_Need']
            dom_uncertainty = sol_j['Total_Uncertainty'] <= sol_i['Total_Uncertainty'] # For minimization

            strictly_better_impact = sol_j['Total_Impact'] > sol_i['Total_Impact']
            strictly_better_need = sol_j['Total_Need'] > sol_i['Total_Need']
            strictly_better_uncertainty = sol_j['Total_Uncertainty'] < sol_i['Total_Uncertainty']

            if (dom_impact and dom_need and dom_uncertainty) and \
               (strictly_better_impact or strictly_better_need or strictly_better_uncertainty):
                is_dominated = True
                break

        if not is_dominated:
            pareto_front_solutions.append(sol_i)

    # Convert to DataFrame and drop duplicates based on objective values (as different epsilon combos might yield same solution)
    pareto_df = pd.DataFrame(pareto_front_solutions).drop_duplicates(
        subset=['Total_Impact', 'Total_Need', 'Total_Uncertainty', 'Total_Cost'] # Include Cost in uniqueness check
    ).reset_index(drop=True)

    print(f"Found {len(pareto_df)} non-dominated solutions for {city_name}.")
    return pareto_df

In [42]:
# Step 6: Main Execution Block for Multi-Objective Optimization
# -------------------------------------------------------------
# Function to load and process survey data
def prepare_survey_data(filepath: str) -> pd.DataFrame:
    """
    Loads the survey data and performs basic preprocessing.
    Returns: Processed DataFrame or None if file not found.
    """
    try:
        df = pd.read_csv(filepath)
        print("--> Successfully loaded survey data file.")
        # Example preprocessing: Convert relevant columns to numeric, handling errors
        # Identify columns that are likely survey questions (e.g., starting with 'q')
        q_columns = [col for col in df.columns if col.startswith('q')]
        for col in q_columns:
            df[col] = pd.to_numeric(df[col], errors='coerce') # Coerce non-numeric values to NaN
        # Drop rows where essential columns might be missing after coercion, if necessary
        # df.dropna(subset=q_columns, inplace=True) # Uncomment if you need to drop rows with NaN in question columns
        return df
    except FileNotFoundError:
        print(f"ERROR: Survey data file not found at {filepath}")
        return None
    except Exception as e:
        print(f"ERROR: Could not process survey data. {e}")
        return None


if __name__ == "__main__":
    print("\n--- Starting Full Analysis ---")

    # Load all survey data once
    all_survey_data = prepare_survey_data(SURVEY_FILEPATH)
    if all_survey_data is None:
        print("Exiting due to failure to load survey data.")
        exit()

    cities_to_analyze = {
        "Jakarta": {"costs": JAKARTA_COSTS},
        "Phnom Penh": {"costs": PHNOM_PENH_COSTS}
    }

    # Store Pareto fronts for plotting/reporting
    all_pareto_fronts_df = pd.DataFrame()

    for city_name, city_info in cities_to_analyze.items():
        print(f"\n==================================================")
        print(f"       Analyzing Multi-Objective for {city_name}      ")
        print(f"==================================================")

        # Load city-specific coefficients
        city_coefs = load_and_process_coefficients(COEF_FILEPATH, city_name)
        if city_coefs is None:
            print(f"Skipping {city_name} due to missing coefficients.")
            continue

        # Filter survey data for the current city
        city_survey_data = all_survey_data[all_survey_data['City'] == city_name]
        if city_survey_data.empty:
            print(f"Skipping {city_name}: No survey data found.")
            continue

        city_costs = city_info['costs']

        # Prepare policy_data dictionary for optimization
        # This dict combines coefficients, calculated need, and defined costs
        city_policy_data = {}
        for term, details in city_coefs['policy_variables'].items():
            if term in city_survey_data.columns and term in city_costs:
                avg_score = city_survey_data[term].mean()
                need_gap = 5 - avg_score # Higher value for higher need (dissatisfaction)
                cost_val = city_costs[term]

                # Ensure impact is positive; otherwise, it won't contribute to aspiration
                impact_val = details['impact'] # Already max(0, estimate) from load_and_process_coefficients
                uncertainty_val = details['uncertainty'] # std_error from load_and_process_coefficients

                city_policy_data[term] = {
                    'desc': details['desc'],
                    'group': details['group'],
                    'impact': impact_val,
                    'need': need_gap,
                    'uncertainty': uncertainty_val,
                    'cost': cost_val
                }
            else:
                print(f"WARNING: Missing data (survey scores or costs) for policy '{term}' in {city_name}. Skipping this policy for optimization.")

        if not city_policy_data:
            print(f"No valid policies prepared for {city_name}. Skipping multi-objective optimization.")
            continue


        # Step 6.1: Get objective ranges to define epsilon values
        print(f"--> Determining objective ranges for {city_name}...")
        ranges = get_objective_ranges(city_policy_data, BASE_BUDGET)

        # Define epsilon value steps
        # Make sure ranges are not (0.0, 0.0) to avoid division by zero or empty linspace
        impact_min, impact_max = ranges['impact_range']
        need_min, need_max = ranges['need_range']
        uncertainty_min, uncertainty_max = ranges['uncertainty_range']

        # Add a small buffer to max values to ensure inclusive range, and steps
        num_steps_need = max(5, int(need_max - need_min) + 1) # At least 5 steps, or more if range is large
        num_steps_uncertainty = max(5, int(uncertainty_max - uncertainty_min) + 1) # At least 5 steps, or more

        # Avoid creating empty linspace if min==max
        epsilon_need_values = np.linspace(need_min, need_max + 0.001, num_steps_need) if need_max > need_min else [need_min]
        epsilon_uncertainty_values = np.linspace(uncertainty_min, uncertainty_max + 0.001, num_steps_uncertainty) if uncertainty_max > uncertainty_min else [uncertainty_min]

        print(f"   Impact Range: ({impact_min:.2f}, {impact_max:.2f})")
        print(f"   Need Range: ({need_min:.2f}, {need_max:.2f}) - Epsilon steps: {len(epsilon_need_values)}")
        print(f"   Uncertainty Range: ({uncertainty_min:.2f}, {uncertainty_max:.2f}) - Epsilon steps: {len(epsilon_uncertainty_values)}")
# Step 6.2: Solve the multi-objective problem using epsilon-constraint
        print(f"--> Solving multi-objective problem for {city_name}...")
        city_pareto_front_df = solve_multi_objective_city(
            city_name=city_name,
            policy_data=city_policy_data,
            budget=BASE_BUDGET,
            epsilon_need_values=epsilon_need_values,
            epsilon_uncertainty_values=epsilon_uncertainty_values
        )

        if not city_pareto_front_df.empty:
            all_pareto_fronts_df = pd.concat([all_pareto_fronts_df, city_pareto_front_df], ignore_index=True)
            print(f"    Total {len(city_pareto_front_df)} unique Pareto optimal solutions found for {city_name}.")

            # Step 6.3: Basic Visualization of Pareto Front (3D Plot)
            # You might need to install plotly: !pip install plotly
            try:
                import plotly.express as px
                print(f"--> Generating 3D Pareto Front plot for {city_name} (interactive HTML).")
                fig = px.scatter_3d(city_pareto_front_df,
                                    x='Total_Impact',
                                    y='Total_Need',
                                    z='Total_Uncertainty',
                                    color='Total_Cost', # Color points by total cost
                                    title=f'Pareto Front for {city_name} (Budget={BASE_BUDGET})',
                                    labels={
                                        'Total_Impact': 'Total Aspiration Impact (Maximize)',
                                        'Total_Need': 'Total Need Addressed (Maximize)',
                                        'Total_Uncertainty': 'Total Uncertainty (Minimize)'
                                    },
                                    hover_data=['Chosen_Policy_Variables_Tuple', 'Total_Cost'])
                fig.update_layout(scene_zaxis_autorange="reversed") # Reverse Z-axis so lower uncertainty is "up"
                fig.write_html(f"{city_name.lower().replace(' ', '_')}_pareto_front_3d.html")
                print(f"    Saved 3D plot to {city_name.lower().replace(' ', '_')}_pareto_front_3d.html")

                # Basic 2D Projections for static plots (e.g., for paper)
                plt.figure(figsize=(18, 6))

                # Impact vs Need
                plt.subplot(1, 3, 1)
                sns.scatterplot(data=city_pareto_front_df, x='Total_Impact', y='Total_Need', hue='Total_Uncertainty', palette='viridis_r', size='Total_Cost', sizes=(20,200), legend='brief')
                plt.title(f'{city_name}: Impact vs Need')
                plt.xlabel('Total Aspiration Impact (Maximize)')
                plt.ylabel('Total Need Addressed (Maximize)')
                plt.grid(True, linestyle='--', alpha=0.6)

                # Impact vs Uncertainty
                plt.subplot(1, 3, 2)
                sns.scatterplot(data=city_pareto_front_df, x='Total_Impact', y='Total_Uncertainty', hue='Total_Need', palette='viridis', size='Total_Cost', sizes=(20,200), legend='brief')
                plt.title(f'{city_name}: Impact vs Uncertainty')
                plt.xlabel('Total Aspiration Impact (Maximize)')
                plt.ylabel('Total Uncertainty (Minimize)')
                plt.gca().invert_yaxis() # Invert Y-axis for uncertainty (lower is better)
                plt.grid(True, linestyle='--', alpha=0.6)

                # Need vs Uncertainty
                plt.subplot(1, 3, 3)
                sns.scatterplot(data=city_pareto_front_df, x='Total_Need', y='Total_Uncertainty', hue='Total_Impact', palette='magma', size='Total_Cost', sizes=(20,200), legend='brief')
                plt.title(f'{city_name}: Need vs Uncertainty')
                plt.xlabel('Total Need Addressed (Maximize)')
                plt.ylabel('Total Uncertainty (Minimize)')
                plt.gca().invert_yaxis() # Invert Y-axis for uncertainty
                plt.grid(True, linestyle='--', alpha=0.6)

                plt.tight_layout()
                plt.savefig(f"{city_name.lower().replace(' ', '_')}_pareto_projections.png", dpi=300)
                print(f"    Saved 2D projections to {city_name.lower().replace(' ', '_')}_pareto_projections.png")
                # plt.show() # Uncomment if you want to display plots in Colab output directly
                plt.close() # Close the plot to free memory

            except ImportError:
                print("Plotly or Seaborn not installed. Skipping advanced plotting.")
                print("Install with: !pip install plotly matplotlib seaborn")
            except Exception as e:
                print(f"Error during plotting for {city_name}: {e}")
                # Step 6.4: Report a few selected Pareto optimal solutions
            print(f"\n--> Selected Pareto Optimal Solutions for {city_name}:")
            # Sort by Impact (desc), then Need (desc), then Uncertainty (asc)
            sorted_pareto = city_pareto_front_df.sort_values(
                by=['Total_Impact', 'Total_Need', 'Total_Uncertainty'],
                ascending=[False, False, True]
            ).reset_index(drop=True)

            if not sorted_pareto.empty:
                # Top Impact Solution
                print("\n--- Solution 1: Max Aspiration Impact ---")
                # Find the solution with the highest impact. If ties, pick one.
                sol1_idx = sorted_pareto['Total_Impact'].idxmax()
                sol1 = sorted_pareto.loc[sol1_idx]
                print(f"    Total Impact: {sol1['Total_Impact']:.4f}, Total Need: {sol1['Total_Need']:.4f}, Total Uncertainty: {sol1['Total_Uncertainty']:.4f}, Total Cost: {sol1['Total_Cost']:.1f}")
                print("    Policies:", [p['Policy'] for p in sol1['Chosen_Policies_Detail']])

                # Find a solution that prioritizes Need (might be the same as top impact, or different)
                # Sort by Need (desc), then Impact (desc), then Uncertainty (asc)
                sorted_by_need = city_pareto_front_df.sort_values(
                    by=['Total_Need', 'Total_Impact', 'Total_Uncertainty'],
                    ascending=[False, False, True]
                ).reset_index(drop=True)
                sol2_idx = sorted_by_need['Total_Need'].idxmax()
                sol2 = sorted_by_need.loc[sol2_idx]
                # Check if this solution is distinct from Solution 1
                if not (sol1['Total_Impact'] == sol2['Total_Impact'] and
                        sol1['Total_Need'] == sol2['Total_Need'] and
                        sol1['Total_Uncertainty'] == sol2['Total_Uncertainty']):
                    print("\n--- Solution 2: Max Need Addressed (if different from Sol 1) ---")
                    print(f"    Total Impact: {sol2['Total_Impact']:.4f}, Total Need: {sol2['Total_Need']:.4f}, Total Uncertainty: {sol2['Total_Uncertainty']:.4f}, Total Cost: {sol2['Total_Cost']:.1f}")
                    print("    Policies:", [p['Policy'] for p in sol2['Chosen_Policies_Detail']])

                # Find a solution that prioritizes minimizing Uncertainty
                # Sort by Uncertainty (asc), then Impact (desc), then Need (desc)
                sorted_by_uncertainty = city_pareto_front_df.sort_values(
                    by=['Total_Uncertainty', 'Total_Impact', 'Total_Need'],
                    ascending=[True, False, False]
                ).reset_index(drop=True)

                # Filter out the 'do nothing' solution if it exists
                meaningful_uncertainty_sols = sorted_by_uncertainty[sorted_by_uncertainty['Total_Cost'] > 0]

                if not meaningful_uncertainty_sols.empty:
                    sol3_idx = meaningful_uncertainty_sols['Total_Uncertainty'].idxmin() # Using idxmin for minimum
                    sol3 = meaningful_uncertainty_sols.loc[sol3_idx]

                    # Check if this solution is distinct from Solutions 1 and 2
                    is_distinct_from_sol1 = not (sol1['Total_Impact'] == sol3['Total_Impact'] and sol1['Total_Need'] == sol3['Total_Need'] and sol1['Total_Uncertainty'] == sol3['Total_Uncertainty'])
                    is_distinct_from_sol2 = not (sol2['Total_Impact'] == sol3['Total_Impact'] and sol2['Total_Need'] == sol3['Total_Need'] and sol2['Total_Uncertainty'] == sol3['Total_Uncertainty'])

                    if is_distinct_from_sol1 and is_distinct_from_sol2:
                        print("\n--- Solution 3: Min Uncertainty (among solutions with policies) ---")
                        print(f"    Total Impact: {sol3['Total_Impact']:.4f}, Total Need: {sol3['Total_Need']:.4f}, Total Uncertainty: {sol3['Total_Uncertainty']:.4f}, Total Cost: {sol3['Total_Cost']:.1f}")
                        print("    Policies:", [p['Policy'] for p in sol3['Chosen_Policies_Detail']])
                    else: # If min uncertainty solution (with policies) is not distinct from Sol 1 or 2
                         print("\n--- Solution 3: Min Uncertainty (among solutions with policies) is not distinct from Sol 1 or 2 ---")
                         print(f"    Total Impact: {sol3['Total_Impact']:.4f}, Total Need: {sol3['Total_Need']:.4f}, Total Uncertainty: {sol3['Total_Uncertainty']:.4f}, Total Cost: {sol3['Total_Cost']:.1f}")
                         print("    Policies:", [p['Policy'] for p in sol3['Chosen_Policies_Detail']])

                else:
                    print("    No feasible solutions with policies found to report for Min Uncertainty.")
            else:
                print("    No Pareto optimal solutions found to report.") # Original else block for empty sorted_pareto

        else:
            print(f"    No Pareto front solutions found for {city_name}.")

    print("\n--- Full Analysis Complete ---")


--- Starting Full Analysis ---
--> Successfully loaded survey data file.

       Analyzing Multi-Objective for Jakarta      
--> Determining objective ranges for Jakarta...
   Impact Range: (0.00, 5.15)
   Need Range: (0.00, 8.48) - Epsilon steps: 9
   Uncertainty Range: (0.00, 0.57) - Epsilon steps: 5
--> Solving multi-objective problem for Jakarta...
Found 7 non-dominated solutions for Jakarta.
    Total 7 unique Pareto optimal solutions found for Jakarta.
--> Generating 3D Pareto Front plot for Jakarta (interactive HTML).
    Saved 3D plot to jakarta_pareto_front_3d.html
    Saved 2D projections to jakarta_pareto_projections.png

--> Selected Pareto Optimal Solutions for Jakarta:

--- Solution 1: Max Aspiration Impact ---
    Total Impact: 5.1536, Total Need: 6.8605, Total Uncertainty: 0.5659, Total Cost: 20.0
    Policies: ['I feel safe in this neighborhood', 'The healthcare facilities in my neighborhood are of good quality', 'I have saved money to invest for my extra income', 'In

/usr/local/lib/python3.11/dist-packages/pulp/pulp.py:1489: UserWarning:

Spaces are not permitted in the name. Converted to '_'



Found 5 non-dominated solutions for Phnom Penh.
    Total 5 unique Pareto optimal solutions found for Phnom Penh.
--> Generating 3D Pareto Front plot for Phnom Penh (interactive HTML).
    Saved 3D plot to phnom_penh_pareto_front_3d.html
    Saved 2D projections to phnom_penh_pareto_projections.png

--> Selected Pareto Optimal Solutions for Phnom Penh:

--- Solution 1: Max Aspiration Impact ---
    Total Impact: 6.3279, Total Need: 8.7497, Total Uncertainty: 0.8233, Total Cost: 20.0
    Policies: ['This neighbourhood is a good place for children', 'The healthcare facilities in my neighborhood are of good quality', 'There are ample employment opportunities within accessible distance', 'I have saved money to invest for my extra income', 'In our neighbourhood neighbours look out for each other', 'I feel that I am not disadvantaged as compared to others']

--- Solution 3: Min Uncertainty (among solutions with policies) ---
    Total Impact: 1.9560, Total Need: 3.5398, Total Uncertainty: 0.